In [ ]:
!pip install langchain

In [4]:
import DBObject
import sqlite3
import generatePrompt
from langchain.utilities import SQLDatabase
from typing import List

ModuleNotFoundError: No module named 'langchain'

In [ ]:
def connect_fun(database_name: str) -> DBObject:
  conn = SQLDatabase.from_uri(f"sqlite://{database_name}?mode=ro") # open in readonly mode
  return DBObject(conn)

In [ ]:

def query_fun(question: str, tables: List[str], database_connection: DBObject) -> str:
  prompt = generatePrompt()

  pass